# LMA: Levenberg-Marquardt Algorithm

The `LMA` operator implements the Levenberg-Marquardt algorithm, an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent.

It is an iterative algorithm. Every iteration, the residual and the Jacobian are calculated for a given set of parameters $p$. Then a new guess $p-\Delta p$ is calculated, such that:

$$(J^T J + \lambda D)\Delta p = J^T y$$

where $J$ is the Jacobian ($J_{ij}=\partial y_i / \partial p_j$), $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon(\lambda), (J^T J)_{kk})$ with $\epsilon(\lambda)$ the damping floor (dependent on the damping factor), $y$ is the residual, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor is decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum numbers of iterations reached
* Residual below specified tolerance
* Relative change in parameters or residual below specified tolerance
* Stagnation with maximum damping factor

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

### Adaptive damping floor

To enhance numerical stability, particularly when dealing with Jacobian matrices $J$ where $J^T J$ may have very small or zero diagonal elements, this LMA implementation employs an adaptive floor for the diagonal elements of the damping scaling matrix $D$. The floor $\epsilon(\lambda)$ is calculated such that it takes the value $\epsilon_0$ (an arbitrarily small number) when $\lambda$ is $\lambda_0$ or lower than $\lambda_0$ (so, $\lambda_{norm}≤1$), and it approaches 1 as $\lambda$ approaches $\lambda_{max}$.

This adaptive mechanism ensures that while a very low floor is used during optimistic (low damping) phases, a more substantial floor (approaching 1) is automatically applied to weak components when high overall damping is required.

## Usage

### `LMA` Operator

    R←{X}f LMA Y

`LMA` is a monadic operator, which takes a left operand to return a derived ambivalent function. This derived function allows to minimize a residual function with a known Jacobian using the Levenberg-Marquadt algorithm, given an initial set of parameters. Several configuration options are available, with sensible defaults previously defined.

The left operand `f` must be a configuration namespace or a function. Configuration namespaces may define the following configuration parameters:

* `toli`: Maximum number of iterations (default `1E3`)
* `tols`: Tolerance for the sum of squared residuals (default `⎕CT`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `dnorm=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)
* `dp`: Perturbation applied to parameters for numerical estimation of the Jacobian (default `⎕CT`)
* `verbose`: If `1`, print `iter ssr rel dnorm p` each iteration (default `0`)

Configuration namespaces may also contain the functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function `Eval` must return either the residuals and the Jacobian for the given set of parameters, or only the residuals. Whenever the residual and Jacobian need to be evaluated, the function `Eval` will be called with trial parameters as right argument and left argument `X`, if given (`Eval` will be called monadically if the derived function `f LMA` is called monadically). `Eval` must return either a two elements vector with the residuals in the first element and the Jacobian in the second one, or a vector of residuals, enclosed if they are not simple scalars. If a Jacobian is not returned, a numerical estimation is calculated evaluating the residual function after applying small perturbations to the parameters (as defined by `dp`, which can be a scalar or a vector).

The `Callback` function will be called every iteration before checking convergence, with a solution namespace for the current guess as right argument and `X` as left argument, if given (`Callback` will be called monadically if the derived function `f LMA` is called monadically). Its return value is discarded.

If `f` is a function, the result is equivalent to using as `f` a namespace with an `Eval` fuction `f`
(with default values for the rest of parameters).

`Y` must be a vector.
The first element of `Y`, or `⊂Y` if `1=≡⍵`, contains the initial guess for the parameters.
If the next element of `Y` is a scalar numeric value, it is interpreted as the initial normalized damping factor.
Additional elements of `Y` must be configuration namespaces. The final configuration parameters are obtained
overwriting the parameters in the namespace given as left operand with those given as right argument from right to left. Default values will be used for non-defined parameters and the `Callback` function, but the `Eval` function must be defined by the user either as left operand `f` or as member of a configuration namespace.

The returned value `R` is a solution namespace corresponding to the last accepted solution.
A solution namespace is a configuration namespace with all the parameters used when running the algorithm and the additional elements:

* `iter`: Number of iterations
* `ssr`: Sum of squared residuals
* `rel`: Relative change metric
* `dnorm`: Normalized damping factor
* `p0`: Initial guess
* `p`: Accepted guess

#### Notes

* With the exception of `toli` and `tols`, configuration parameters should be modified only by expert users or in case of convergence problems

* The relative change metric `rel` is the minimum relative change between successive accepted solutions either in the the sum of squared residuals or in the parameters themselves

* In addition to being used for the definition of default values, `⎕CT` is also the baseline for adaptive floor damping


### `LM` Operator

`LM` is a simplified version of `LMA`. It is a dyadic operator from which a dydadic function is derived. Usage:

    R←X f LM g Y

where `f` is a monadic function which returns the residuals and Jacobian given a set of parameters, `g` is a monadic function which takes as argument an `iter ssr rel dnorm p` vector and gets called before every convergence check, `Y` is a two elements vector with the initial guess of parameters and normalized damping factor, and `X` is a vector with the configuration parameters `toli tols tolr tolg dini dinc ddec dmax dmin`. The return value `R` will be an `iter ssr rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
]dinput
LM←{ti ts tr tg d0 di dd dx dn←⍺ ⋄ p d←⍵               ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢                     ⍝ damping factor normalization(λ)
    E←⍺⍺{s←+.×⍨⊃y j←⍺⍺ ⍵ ⋄ s ⍵ y j}                    ⍝ eval(p): sum(error²), parameters, residual, jacobian
    A←{s p y j←⍵ ⋄ s p,(t+.×j)(y+.×⍨t←⍉j)}             ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ b←1-÷1⌈d⊢←D ⍵             ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←(⎕CT+b-⎕CT×b)⌈1 1⍉jj        ⍝     change of parameters with adaptive floor
        s0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        s1←s-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|s1÷s                      ⍝     relative change in parameters or residuals
        (⎕CT>s0)∧⎕CT<s1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤s0)∧tg≥s1÷s0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣s p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)s r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(ts>s)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ _←⍵⍵ i s r d p⊣s p jj jy←A E p             ⍝ init
    i s r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ return iterations, ssr, change, norm damping, parameters
}

In [3]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ 0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('isrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'ddec' 'dmin'                                   ⍝ computed defaults
    nr←'iter' 'ssr' 'rel' 'dnorm' 'p'                  ⍝ results
    c←(                                                ⍝ default config
        toli:1000 ⋄ tols:⎕CT ⋄ tolr:⎕CT ⋄ tolg:0.01    ⍝     tolerances
        dini:0.01 ⋄ dinc:5 ⋄ dmax:÷⎕CT                 ⍝     damping
        p0:p ⋄ dp:⎕CT ⋄ verbose:0                      ⍝     init guess, perturbation, logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵ ⋄ ⍵}        ⍝ format and print
    D←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                         ⍝ display form
    J←{⍉↑⍺÷⍨(⍺⍺¨(⊂⍵)+↓↑(-⍳≢⍵)↑¨⍺)-⊆⍺⍺ ⍵}               ⍝ estimate Jacobian

    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←c ⎕NS ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    _←c ⎕VSET(↑nc)(c ⎕VGET(↑nc)c.(÷dinc dmax))         ⍝ computed defaults
    C←⍺{⍺⍺ c.CallBack c ⎕VSET(↑nr)⍵}P⍣(c.verbose≢0)    ⍝ callback function
    E←⍺∘c.Eval{(1<≡e)∧2=≢e←⍺⍺ ⍵:e ⋄ (⊃⊆e)(⍺ ⍺⍺ J ⍵)}   ⍝ eval function
    c⊣c.⎕DF D(c ⎕VGET↑n)(c.dp∘E)LM C p c.dnorm         ⍝ return namespace
}

### Tests

In [4]:
]dinput
Test←{0::⎕EM ⎕SIGNAL ⎕EN
    t←()
    ⍝ Tests ability to navigate sharp, narrow valleys and handle situations where the Jacobian
    ⍝ may lead to a locally rank-deficient J^T J matrix (eg. zero diagonal elements), requiring
    ⍝ robust damping
    t.Beale←{⎕CT←1e¯10
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)[(¯1+y)x ⋄ (¯1+y*2)(2×x×y) ⋄ (¯1+y*3)(3×x×y*2)]
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).ssr>(R B #.LMA 2 2).ssr             ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).ssr>(R B #.LMA ¯1 1).ssr           ⍝ closer to another local minimum
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.BealeNum←{⎕CT←1e¯10
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).ssr>(R B #.LMA 2 2).ssr             ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).ssr>(R B #.LMA ¯1 1).ssr           ⍝ closer to another local minimum
        r
    }
    ⍝ Assesses capability to follow a long, narrow, curving multi-dimensional valley, testing
    ⍝ the interplay between step length and direction adjustments controlled by the damping factor
    t.Helical←{
        T←{1|1+(12○⍺+0j1×⍵)÷○2} ⋄ M←{|⍺+0j1×⍵}
        H←{
            a b c←⍵ ⋄ m←a M b ⋄ y←(10×c-10×a T b)(10×m-1)c
            y[((50×b)÷○+.×⍨a b)(-(50×a)÷○+.×⍨a b)10 ⋄ (10×a÷m)(10×b÷m)0 ⋄ 0 0 1]
        }
        r←1 0 0 CMP(R H #.LMA¯1 0 0).p                                ⍝ standard starting point
        r,←1 0 0 CMP(R H #.LMA¯1.2 0.1 0.1).p                         ⍝ slightly perturbed
        r,←1 0 0 CMP(R H #.LMA¯0.9 ¯0.05 ¯0.05).p                     ⍝ in other direction
        r,←1 0 0 CMP(R H #.LMA 0.5 ¯0.5 0.5).p                        ⍝ qualitatively different
        r,←1 0 0 CMP(R H #.LMA¯0.5 0.5 ¯0.5).p                        ⍝ another one
        r,←1 0 0 CMP(R H #.LMA¯1 0 10).p                              ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA¯1 0 ¯10).p                             ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA 3 4 5).p                               ⍝ away in all components
        r
    }
    ⍝ Verifies efficiency and correctness for a linear system. Converge shoud be very fast
    ⍝ (1 or 2 iterations) with minimal damping, demonstrating Gauss-Newton like behavior
    t.Linear←{
        y←(A←?100 10⍴0)+.×x←⍳10 ⋄ s←R{(y-⍨A+.×⍵)A}#.LMA(?10⍴0)0 ⋄ (1=s.iter),(⍳10)CMP s.p
    }
    ⍝ numerical approximation of the Jacobian
    t.LinearNum←{
        y←(A←?100 5⍴0)+.×x←⍳5 ⋄ s←R{y-⍨A+.×⍵}#.LMA(?5⍴0)0 ⋄ (⍳5)CMP s.p
    }
    ⍝ Evaluates LMA's robustness and ability to converge to a solution when the Jacobian
    ⍝ becomes singular (rank-deficient) at or near the optimum
    t.Powell←{
        P←{
            a b c d←⍵ ⋄ r5 r10←5 10*÷2 ⋄ y←(a+10×b)(r5×c-d)(×⍨b-2×c)(r10××⍨a-d)
            y[1 10 0 0 ⋄ 0 0 r5(-r5) ⋄ 0(2×b-2×c)(-2×b-2×c)0 ⋄ (2×r10×a-d)0 0(-2×r10×a-d)]
        }
        r←(4⍴0)CMP(R P #.LMA(3 ¯1 0 1)(tols:1e¯30)).p                 ⍝ standard starting point
        r,←(4⍴0)CMP(R P #.LMA(0 0 0 0)(tols:1e¯30)).p                 ⍝ solution (convergence at zero step)
        r,←(4⍴0)CMP(R P #.LMA(1 1 1 1)(tols:1e¯30)).p                 ⍝ far from solution
        r
    }
    ⍝ Tests LMA's performance in minimizing a classic non-linear function characterized by
    ⍝ a deep, narrow, banana-shaped valley, requiring effective adaptation of search
    ⍝ direction and step size
    Rosenbrock←{p q←⍵ ⋄ ((10×q-×⍨p),1-p)[(-20×p)10 ⋄ ¯1 0]}
    t.Rosenbrock←{
        r←1 1 CMP(R #.Rosenbrock #.LMA 1.5 1.5).p                     ⍝ close to the solution
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 1).p                        ⍝ not so close
        r,←1 1 CMP(R #.Rosenbrock #.LMA 0 0).p                        ⍝ outside of parabollic valley
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯1.2 1).p                     ⍝ further
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯2 ¯2).p                      ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 2).p                        ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.RosenbrockNum←{
        r←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 1.5 1.5).p                    ⍝ close to the solution
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 1).p                       ⍝ not so close
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 0 0).p                       ⍝ outside of parabollic valley
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯1.2 1).p                    ⍝ further
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯2 ¯2).p                     ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 2).p                       ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ Check that algorithm finishes on termination conditions
    t.Terminate←{
        r←(R #.Rosenbrock #.LMA(0 0)(tols:0 ⋄ tolr:0)).(iter>toli)              ⍝ number of iterations
        r,←(R #.Rosenbrock #.LMA(0 0)(tols:0)).(rel<tolr)                       ⍝ relative change
        r,←(R #.Rosenbrock #.LMA(0 0)(tols:0 ⋄ dmax:1e6)).((dnorm>1e6)∧rel=0)   ⍝ maximum damping
        r
    }
    ⍺←1e¯6 ⋄ tol←⍺ ⋄ CMP←{tol>|⍺-⍵} ⋄ R←{⎕←(10↑''),⍵.((6 0⍕iter),(12 ¯5⍕ssr),p) ⋄ ⍵}
    (t⎕NS'tol' 'CMP' 'R'){⎕←⍵ ⋄ 0∊⍺⍎⍵,'⍬':('TEST FAILED: ',⍵)⎕SIGNAL 8 ⋄ _←0}¨(↓t.⎕NL 3)⊣⍣(⍵≡'*')⊆⍵
}

In [5]:
Test'*'

Beale 
 6 6.8539E¯21 3 0.4999999999
 9 2.9291E¯21 3 0.5
 32 6.2363E¯17 2.99999998 0.4999999949
 8 5.2033E¯19 3 0.5
 842 4.5211E¯1 ¯15356.11978 1.000064535
 840 4.5211E¯1 ¯15356.14972 1.000064535
BealeNum 
 8 9.8409E¯18 3 0.5000000006
 9 9.8960E¯15 2.999999964 0.5000000115
 33 2.4618E¯16 2.999999991 0.5000000011
 9 9.0815E¯18 2.999999996 0.4999999996
 66 6.0792E¯1 ¯9.386917068 1.096824751
 56 5.5962E¯1 ¯13.7892706 1.067654204
Helical 
 10 1.1919E¯19 1 2.177303483E¯10 3.448140016E¯10
 10 1.1846E¯19 1 2.170636163E¯10 3.437591012E¯10
 8 4.6695E¯19 1 4.309357625E¯10 6.82517468E¯10
 14 4.6833E¯20 1 1.364817538E¯10 2.161410116E¯10
 12 1.5718E¯20 1 7.90691E¯11 1.252178359E¯10
 24 2.7419E¯20 1 1.044305611E¯10 1.65382662E¯10
 21 8.5502E¯15 1 5.82997047E¯8 9.234613263E¯8
 10 3.6281E¯20 1 1.201260465E¯10 1.902399678E¯10
Linear 
 1 1.8514E¯24 1 2 3 4 5 6 7 8 9 10
LinearNum 
 7 2.0406E¯14 1.000000012 1.999999985 3.000000028 3.999999987 4.99999997
Powell 
 99 9.8312E¯31 2.241601808E¯8 ¯2.241601808E¯9 1.446057221E¯8 1.446057221E¯8
 1 0.0000E0 0 0 0 0
 97 9.8174E¯31 ¯2.241693431E¯8 2.241693431E¯9 ¯1.445422286E¯8 ¯1.445422286E¯8
Rosenbrock 
 8 7.4732E¯15 1.000000086 1.00000017
 12 7.8195E¯15 1.000000088 1.000000174
 32 5.1271E¯15 0.999999929 0.999999857
 28 4.8548E¯15 0.9999999309 0.9999998609
 18 5.1595E¯15 0.9999999287 0.9999998566
 14 8.0558E¯15 1.000000089 1.000000176
RosenbrockNum
 12 1.5737E¯14 1.000000069 1.000000127
 14 4.5525E¯15 1.000000045 1.000000085
 33 1.2191E¯20 1 1
 28 2.4615E¯15 0.9999999589 0.9999999151
 16 1.7653E¯15 0.9999999642 0.9999999262
 19 1.8684E¯14 1.000000074 1.000000137
Terminate 
 1001 0.0000E0 1 1
 34 1.4249E¯29 1 1
 13 1.0000E0 0 0

In [6]:
] _←link.export -overwrite # APLSource

## Examples

In [7]:
NOISY←{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}

### Exponential decay

In [8]:
ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵} ⋄ ExpDecEv←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
p←10 0.5 1 ⋄ y←0.25 NOISY p ExpDec x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p ExpDec x)(⊃x y ExpDecEv p)
1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT(x y∘ExpDecEv)LM⊢(5 0.1 0.5)1 ⍝ ti ts tr tg d0 di dd dx dn
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(verbose:1)

┌──────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 10.92911 6.95210 4.73781 3.31780 2.33215 1.77329 1.51348 1.22164 1.13532 1.18606│
│ 11.00000 7.06531 4.67879 3.23130 2.35335 1.82085 1.49787 1.30197 1.18316 1.11109│
│ ¯0.07089 ¯0.11321 0.05902 0.08649 ¯0.02120 ¯0.04756 0.01561 ¯0.08034 ¯0.04784 0.07497│
└──────────────────────────────────────────────────────────────────────────────────────┘

┌─┬─────────────┬───────────────┬──────────────┬────────────────────────────────────┐
│6│0.03578835187│3.382192234E¯19│0.000319999999│9.927273403 0.4929284896 0.977406695│
└─┴─────────────┴───────────────┴──────────────┴────────────────────────────────────┘

0 4.9910E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.3805E1 2.6184E¯1 1.0000E0 6.5107E0 3.7739E¯1 2.5627E0 
 2 2.4937E¯1 2.5207E¯1 2.0000E¯1 9.7272E0 5.4444E¯1 1.1473E0 
 3 4.0897E¯2 6.2711E¯4 4.0000E¯2 9.9097E0 4.9014E¯1 9.9209E¯1 
 4 3.5788E¯2 5.3168E¯6 8.0000E¯3 9.9272E0 4.9293E¯1 9.7746E¯1 
 5 3.5788E¯2 7.8368E¯11 1.6000E¯3 9.9273E0 4.9293E¯1 9.7741E¯1 
 6 3.5788E¯2 3.3822E¯19 3.2000E¯4 9.9273E0 4.9293E¯1 9.7741E¯1 
 s: 3.5788E¯2 p: 9.9273E0 4.9293E¯1 9.7741E¯1

In [9]:
⊢ed←x y ExpDecEv LMA⊂5 0.1 0.5
x y(ed LMA)ed.p0

s: 3.5788E¯2 p: 9.9273E0 4.9293E¯1 9.7741E¯1

s: 3.5788E¯2 p: 9.9273E0 4.9293E¯1 9.7741E¯1

In [10]:
x y(ed LMA)ed.p0(verbose:1)

0 4.9910E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.3805E1 2.6184E¯1 1.0000E0 6.5107E0 3.7739E¯1 2.5627E0 
 2 2.4937E¯1 2.5207E¯1 2.0000E¯1 9.7272E0 5.4444E¯1 1.1473E0 
 3 4.0897E¯2 6.2711E¯4 4.0000E¯2 9.9097E0 4.9014E¯1 9.9209E¯1 
 4 3.5788E¯2 5.3168E¯6 8.0000E¯3 9.9272E0 4.9293E¯1 9.7746E¯1 
 5 3.5788E¯2 7.8368E¯11 1.6000E¯3 9.9273E0 4.9293E¯1 9.7741E¯1 
 6 3.5788E¯2 3.3822E¯19 3.2000E¯4 9.9273E0 4.9293E¯1 9.7741E¯1 
 s: 3.5788E¯2 p: 9.9273E0 4.9293E¯1 9.7741E¯1

In [11]:
⊢s←x y{x y←⍺ ⋄ y-⍵ ExpDec x}LMA(5 0.1 0.5)

s: 3.5789E¯2 p: 9.9275E0 4.9297E¯1 9.7744E¯1

### Linear system

In [12]:
Linear←{a b←⍺ ⋄ a+b×⍵} ⋄ LinearEv←{(y-⍵ Linear x)(⍉↑-(=⍨x)x)}
p←0.1 2 ⋄ y←1e¯3 NOISY p Linear x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Linear x)(⊃LinearEv p)
LinearEv LMA(0 0)(dinc:1e3 ⋄ verbose:1)

┌────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 0.09959 2.09998 4.10008 6.10035 8.09961 10.10012 12.09971 14.09960 16.09993 18.09962│
│ 0.10000 2.10000 4.10000 6.10000 8.10000 10.10000 12.10000 14.10000 16.10000 18.10000│
│ ¯0.00041 ¯0.00002 0.00008 0.00035 ¯0.00039 0.00012 ¯0.00029 ¯0.00040 ¯0.00007 ¯0.00038│
└────────────────────────────────────────────────────────────────────────────────────────┘

0 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 1 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 2 1.1581E3 0.0000E0 1.0000E3 0.0000E0 0.0000E0 
 3 1.1581E3 0.0000E0 1.0000E6 0.0000E0 0.0000E0 
 4 1.1581E3 0.0000E0 1.0000E9 0.0000E0 0.0000E0 
 5 1.1581E3 3.4300E¯10 1.0001E12 9.0999E¯10 2.0158E¯10 
 6 1.1581E3 3.4300E¯7 1.0000E9 9.1090E¯7 2.0178E¯7 
 7 1.1577E3 3.4291E¯4 1.0000E6 9.1071E¯4 2.0174E¯4 
 8 8.4792E2 2.6756E¯1 1.0000E3 7.5749E¯1 1.7256E¯1 
 9 3.2731E¯1 9.9961E¯1 1.0000E0 3.8705E¯1 1.9370E0 
 10 6.1194E¯7 2.2137E¯2 1.0000E¯3 9.9984E¯2 2.0000E0 
 11 6.1070E¯7 1.0033E¯10 1.0000E¯6 9.9964E¯2 2.0000E0 
 12 6.1070E¯7 4.1117E¯25 9.9900E¯10 9.9964E¯2 2.0000E0 
 s: 6.1070E¯7 p: 9.9964E¯2 2.0000E0

### Gauss function

In [13]:
Gauss←{a s m c←⍺ ⋄ c+a×m s X ⍵} ⋄ X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)}
GaussEv←{x y←⍺ ⋄ a s m c←⍵ ⋄ x2←(x1←(xe←m s X x)×a×(x-m)÷×⍨s)×(x-m)÷s ⋄ (y-⍵ Gauss x)(⍉↑(-xe)x1(-x2)(-=⍨xe))}
p←10 5 1.5 2 ⋄ y←0 NOISY p Gauss x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Gauss x)(⊃x y GaussEv p)
(x y GaussEv LMA(8 4 1 1)(dini:1e¯3 ⋄ tols:1e¯6)).(iter ssr rel dnorm p)
(x y GaussEv LMA(8 4 1 1)1e¯1(tols:1e¯6)).(iter ssr rel dnorm p)
(x y GaussEv LMA(8 4.5 1.2 1.5)(tolr:1e¯3)).(iter ssr rel dnorm p)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬──────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│1220703140│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴──────────┴───────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬─────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│122070314│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴─────────┴───────────────────────────────────────────────┘

┌─┬────────────┬───────────────┬──┬───────────────────────────────────────────────┐
│7│0.9170106519│0.0007211535235│25│9.471886955 4.249297788 2.168664717 2.526269831│
└─┴────────────┴───────────────┴──┴───────────────────────────────────────────────┘